In [1]:
import pickle as pkl
import numpy as np
import gzip
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import time

from utils import asMinutes, timeSince, load_zipped_pickle, corpus_bleu, directories
from langUtils import loadLangPairs, langDataset, langCollateFn, initHybridEmbeddings, tensorToList

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Import Dictionaries and Data

In [2]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pkl.load(f)
        return loaded_object

In [3]:
# Print sentence given numbers
def ids2sentence(sentence, dictionary):
    return ' '.join([dictionary[i] for i in sentence])
#ids2sentence(en_train_num[0], id2word_en_dic)

def add_symbol(id2word_dic, word2id_dic):
    symbols = ['<pad>', '<unk>', '<sos>', '<eos>']
    for i, symbol in enumerate(symbols):
        id2word_dic[i] = symbol
        word2id_dic[symbol] = i
    return id2word_dic, word2id_dic

In [4]:
# id2word_vi_dic = load_zipped_pickle("../embeddings/id2word_vi_dic.p")
# word2id_vi_dic = load_zipped_pickle("../embeddings/word2id_vi_dic.p")

# id2word_en_dic = load_zipped_pickle("../embeddings/id2word_en_dic.p")
# word2id_en_dic = load_zipped_pickle("../embeddings/word2id_en_dic.p")

# id2word_vi_dic, word2id_vi_dic = add_symbol(id2word_vi_dic, word2id_vi_dic)
# id2word_en_dic, word2id_en_dic = add_symbol(id2word_en_dic, word2id_en_dic)

# vi_train = load_zipped_pickle("../data/vi-en-tokens/train_vi_tok.p")
# en_train = load_zipped_pickle("../data/vi-en-tokens/train_en_tok.p") # Already Processed for symbols

# vi_train_num = load_zipped_pickle("../data/vi-en-tokens/train_vi_tok_num.p")
# en_train_num = load_zipped_pickle("../data/vi-en-tokens/train_en_tok_num.p") # Already Processed for symbols

In [5]:
vi, en = loadLangPairs("vi")
BATCH_SIZE = 32
train_dataset = langDataset([(vi.train_num[i], en.train_num[i]) for i in range(len(vi.train_num)) if (len(vi.train[i]) < vi.max_length) & (len(en.train[i]) < en.max_length)])
overfit_dataset = langDataset([(vi.train_num[i], en.train_num[i]) for i in range(2)])
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=langCollateFn,
                                           shuffle=True)
dev_dataset = langDataset([(vi.dev_num[i], en.dev_num[i]) for i in range(len(vi.dev_num)) if (len(vi.dev[i]) < vi.max_length) & (len(en.dev[i]) < en.max_length)])
dev_loader = torch.utils.data.DataLoader(dataset=dev_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=langCollateFn,
                                           shuffle=True)

In [6]:
vi.max_length

42

In [7]:
SPECIAL_SYMBOLS_ID = PAD_ID, UNK_ID, SOS_ID, EOS_ID = 0, 1, 2, 3

## Padding Data

### Sort by input data length

In [8]:
# def sort_by_length(data_input, target_data):
#     input_size = [len(data) for data in data_input]
#     size_index = np.argsort(input_size)
#     return list(np.array(data_input)[size_index]), list(np.array(target_data)[size_index])

# vi_train_num, en_train_num = sort_by_length(vi_train_num, en_train_num)

### Padding Data given batch size

In [9]:
def pad(data, length):
    for i, line in enumerate(data):
        if len(line) < length:
            for i in range(len(line), length):
                line.append(0)
        else:
            data[i] = line[0:length]
    return data

# Return the batch data and target
def get_batch(i, batch_size, train_data, train_target):
    if i * batch_size > len(train_data):
        raise Exception('Incorrect batch index')
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    batch_data = list(np.array(train_data)[start_idx:end_idx])
    batch_target = list(np.array(train_target)[start_idx:end_idx])
    batch_data = pad(batch_data, len(batch_data[batch_size - 1]))
    max_target = max([len(data) for data in batch_data])
    batch_target = pad(batch_target, max_target)
    return batch_data, batch_target

# get_batch(5, 64, vi_train_num, en_train_num)

## Models

### Encoder

In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, batch_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        # input_size: input dictionary size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.num_layers = num_layers
        self.gru = nn.GRU(hidden_size, 
                          hidden_size, 
                          num_layers= num_layers, 
                          batch_first = True) # BATCH FIRST

    def forward(self, encoder_input, hidden_input):
        # encoder_input: batch * 1 (for 1 word each time)
        embedded_input = self.embedding(encoder_input)
        # embedded_input: batch * 1 * emb_dim
        # hidden_input: batch * 1(layer) * hidden_size
        output, hidden = self.gru(embedded_input, hidden_input)
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.num_layers, self.batch_size, self.hidden_size, device=device)

### Decoder

In [11]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, batch_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        # output_size: input dictionary size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, 
                          hidden_size,
                          num_layers= num_layers, 
                          batch_first = True) # BATCH_FRIST
        self.out = nn.Linear(hidden_size, output_size)
        # self.softmax = nn.LogSoftmax(dim=1) # Use cross entropy loss outside

    def forward(self, decoder_input, hidden_input):
        # decoder_input: batch * 1
        embedded_input = self.embedding(decoder_input)
        # embedded_input: batch * 1 * emb_dim
        embedded_input = F.relu(embedded_input)
        # hidden_input: batch * hidden_size
        output, hidden = self.gru(embedded_input, hidden_input)
        output = self.out(output)
        # output = self.softmax(output) # not using softmax
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.num_layers, self.batch_size, self.hidden_size, device=device)

## Training

In [12]:
def train(inp, output, out_max, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, batch_size):
    total_avg_loss = 0
    loss = 0
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_len = inp.shape[1]
    encoder_outputs = torch.zeros(input_len, batch_size, 1, HIDDEN_SIZE, device=device)
    encoder_hiddens = torch.zeros(input_len, 1, batch_size, HIDDEN_SIZE, device=device)
    # Encode
    for ec_idx in range(input_len):
        # input batch_size * 1
        encoder_output, encoder_hidden = encoder(inp[:, ec_idx].unsqueeze(1), encoder_hidden)
        encoder_outputs[ec_idx] = encoder_output
        encoder_hiddens[ec_idx] = encoder_hidden

    # Decode
    decoder_input = torch.tensor([SOS_ID] * batch_size, device=device)
    decoder_hidden = encoder_hidden

    # Always use Teacher Forcing
    for dc_idx in range(out_max):
        decoder_output, decoder_hidden = decoder(decoder_input.unsqueeze(1), decoder_hidden)
        decoder_output = decoder_output.squeeze(1).to(device) # get rid of the seq dimention
        loss += criterion(decoder_output, output[:, dc_idx])
        decoder_input = output[:, dc_idx]

        ## Print Value
#         sample_sentence.append(torch.argmax(decoder_output[0]).item())

    loss.backward()
    total_avg_loss += loss.item() / out_max

    encoder_optimizer.step()
    decoder_optimizer.step()

    ## Print Value
#     print("Predict: ", ids2sentence(sample_sentence, en.id2word))
#     print("Actual: ", ids2sentence(output[0].cpu().numpy(), en.id2word))
        
    return total_avg_loss
#     return 0

In [58]:
def bleuEval(encoder, decoder, data_loader, batch_size):
    with torch.no_grad():
        true_outputs = []
        decoder_outputs = []
        for i, (inp, inp_lens, output, out_mask, out_max) in enumerate(train_loader):
            if i * batch_size >= 10000 or len(inp[0]) != batch_size:
                continue
            inp = inp.transpose(0,1).to(device)
            output = output.transpose(0,1).to(device)
            true_outputs.append([[str(tok.item()) for tok in out if tok != 0] for out in output])
            encoder_hidden = encoder.initHidden()
            input_len = inp.shape[1]
            encoder_outputs = torch.zeros(input_len, batch_size, 1, HIDDEN_SIZE, device=device)
            encoder_hiddens = torch.zeros(input_len, 1, batch_size, HIDDEN_SIZE, device=device)

            # Encode
            for ec_idx in range(input_len):
                # input batch_size * 1
                encoder_output, encoder_hidden = encoder(inp[:, ec_idx].unsqueeze(1), encoder_hidden)
                encoder_outputs[ec_idx] = encoder_output
                encoder_hiddens[ec_idx] = encoder_hidden

            # Decode
            decoder_input = torch.tensor([SOS_ID] * batch_size, device=device)
            decoder_hidden = encoder_hidden

            # Greedy
            for dc_idx in range(out_max):
                decoder_output, decoder_hidden = decoder(decoder_input.unsqueeze(1), decoder_hidden)
                decoder_output = decoder_output.squeeze(1).to(device) # get rid of the seq dimention
                topv, topi = decoder_output.topk(1)
                decoder_input = torch.LongTensor([topi[i][0] for i in range(inp.size(0))]).to(device)
                ## Print Value
                decoder_outputs.append(list(decoder_input.cpu().numpy()))
            ## Print Value
        predict = []
        for seq in np.array(decoder_outputs).T.astype(str):
            seq_toks = []
            for tok in seq:
                seq_toks.append(tok)
                if tok == '3':
                    break
            predict.append(seq_toks)
        print('Sample True: ', ' '.join([en.id2word[int(i)] for i in true_outputs[0][0]]))
        print('Sample Predicted: ', ' '.join([en.id2word[int(i)] for i in predict[0]]))
        bleu_score = corpus_bleu(predict, true_outputs, 4)
        return bleu_score

In [59]:
def fit(train_loader, dev_loader, encoder, decoder, encoder_opt, decoder_opt, criterion, batch_size, epochs, print_every):
    start = time.time()
    print('Initializing Model Training + Eval...')
    losses = []
    train_scores = []
    dev_scores = []
    for epoch in range(epochs):
        loss = 0
        for i, (inp, inp_lens, output, out_mask, out_max) in enumerate(train_loader):
            if (len(inp[0]) != batch_size):
                continue
            inp.transpose_(0,1)
            output.transpose_(0,1)
            inp = inp.to(device)
            output = output.to(device)
            loss += train(inp, output, out_max, encoder, decoder, encoder_opt, decoder_opt, criterion, batch_size)
            if i % print_every == 0 and i > 0:
                losses.append(loss/i)
                print("Time Elapsed: {} | Loss: {:.4}".format(asMinutes(time.time() - start),
                                                                                loss/i))
        train_score = bleuEval(encoder, decoder, train_loader, batch_size)
        dev_score = bleuEval(encoder, decoder, dev_loader, batch_size)
        train_scores.append(train_score)
        dev_scores.append(dev_score)
        print("Epoch: {} | Time Elapsed: {} | Loss: {:.4} | Train BLEU: {:.4} | Dev BLEU: {:.4}".format(epoch + 1, 
                                                                                                        asMinutes(time.time() - start),
                                                                                                        loss/len(train_loader), 
                                                                                                        train_score, 
                                                                                                        dev_score))

In [15]:
# dic_size_vi = len(id2word_vi_dic.keys())
# dic_size_en = len(id2word_en_dic.keys())
HIDDEN_SIZE = 128
learning_rate = 0.01

## Add ignore index
criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)

encoder = EncoderRNN(input_size = vi.n_words, hidden_size = HIDDEN_SIZE, num_layers = 1, batch_size = BATCH_SIZE).to(device)
decoder = DecoderRNN(hidden_size = HIDDEN_SIZE, output_size = en.n_words, num_layers = 1, batch_size = BATCH_SIZE).to(device)

encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

In [16]:
fit(train_loader, dev_loader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, BATCH_SIZE, 10, 300)

Initializing Model Training + Eval...
Time Elapsed: 4m 13s | Loss: 5.741
Time Elapsed: 8m 16s | Loss: 5.55
Time Elapsed: 12m 10s | Loss: 5.456
Time Elapsed: 16m 24s | Loss: 5.368
Time Elapsed: 20m 26s | Loss: 5.299
Time Elapsed: 24m 18s | Loss: 5.24
Time Elapsed: 28m 30s | Loss: 5.197
Time Elapsed: 32m 36s | Loss: 5.156
Time Elapsed: 36m 25s | Loss: 5.124
Time Elapsed: 40m 37s | Loss: 5.094
Time Elapsed: 44m 45s | Loss: 5.067
Epoch: 1 | Time Elapsed: 46m 32s | Loss: 5.054 | Train BLEU: 1.563 | Dev BLEU: 1.069
Time Elapsed: 50m 37s | Loss: 4.608
Time Elapsed: 54m 48s | Loss: 4.597
Time Elapsed: 58m 31s | Loss: 4.575
Time Elapsed: 62m 43s | Loss: 4.55
Time Elapsed: 66m 54s | Loss: 4.54
Time Elapsed: 70m 39s | Loss: 4.529
Time Elapsed: 74m 51s | Loss: 4.518
Time Elapsed: 79m 3s | Loss: 4.51
Time Elapsed: 82m 48s | Loss: 4.501
Time Elapsed: 86m 58s | Loss: 4.493
Time Elapsed: 91m 10s | Loss: 4.486
Epoch: 2 | Time Elapsed: 93m 10s | Loss: 4.481 | Train BLEU: 2.315 | Dev BLEU: 2.702
Time Ela

In [60]:
bleuEval(encoder, decoder, train_loader, BATCH_SIZE)

Sample True:  <sos> a few days after arriving , i met this wonderful , old conductor who started casting me in all of these roles . <eos>
Sample Predicted:  <sos> i want to be able to do the truth , i want to be able to do the truth . <eos>


5.984592578474465

In [61]:
pkl.dump(encoder, open("./model/encoder.p", "wb"))
pickle.dump(decoder, open("./model/decoder.p", "wb"))

NameError: name 'pickle' is not defined

In [44]:
encoder.gru.batch_fi

128

In [40]:
%debug

> /opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py(143)check_hidden_size()
    141         def check_hidden_size(hx, expected_hidden_size, msg='Expected hidden size {}, got {}'):
    142             if tuple(hx.size()) != expected_hidden_size:
--> 143                 raise RuntimeError(msg.format(expected_hidden_size, tuple(hx.size())))
    144 
    145         if self.mode == 'LSTM':



ipdb>  encoder


*** NameError: name 'encoder' is not defined


ipdb>  exit


In [34]:
%debug

> <ipython-input-31-f7ca4d53521a>(15)train()
     13         # input batch_size * 1
     14         encoder_output, encoder_hidden = encoder(inp[:, ec_idx].unsqueeze(1), encoder_hidden)
---> 15         encoder_outputs[ec_idx] = encoder_output
     16         encoder_hiddens[ec_idx] = encoder_hidden
     17 



ipdb>  enoder_outputs


*** NameError: name 'enoder_outputs' is not defined


ipdb>  encoder_outputs


tensor([[[[ 0.0170,  0.1717,  0.3893,  ...,  0.0381, -0.1810, -0.2559]],

         [[ 0.0170,  0.1717,  0.3893,  ...,  0.0381, -0.1810, -0.2559]],

         [[ 0.0170,  0.1717,  0.3893,  ...,  0.0381, -0.1810, -0.2559]],

         ...,

         [[ 0.0170,  0.1717,  0.3893,  ...,  0.0381, -0.1810, -0.2559]],

         [[ 0.0170,  0.1717,  0.3893,  ...,  0.0381, -0.1810, -0.2559]],

         [[ 0.0170,  0.1717,  0.3893,  ...,  0.0381, -0.1810, -0.2559]]],


        [[[-0.0219, -0.0105, -0.2507,  ...,  0.1992, -0.4429, -0.4246]],

         [[-0.0108,  0.4253,  0.3838,  ..., -0.2613,  0.1234, -0.1874]],

         [[-0.2046, -0.4536,  0.5676,  ...,  0.0421,  0.3484, -0.3940]],

         ...,

         [[-0.0036, -0.4598,  0.2498,  ..., -0.1157,  0.1377,  0.2163]],

         [[-0.0036, -0.4598,  0.2498,  ..., -0.1157,  0.1377,  0.2163]],

         [[-0.0856,  0.0131,  0.2655,  ...,  0.2046, -0.2525, -0.0867]]],


        [[[ 0.4918, -0.1363,  0.1370,  ...,  0.0146, -0.2607, -0.5579]],

    

ipdb>  len(encoder_outputs)


23


ipdb>  encoder_outputs[23]


*** IndexError: index 23 is out of bounds for dimension 0 with size 23


ipdb>  exit


In [12]:
# fit(overfit_vi_train, overfit_en_train, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, BATCH_SIZE, 100)

Predict:  ['<sos>', '<sos>', '<sos>', 'the', 'the', 'the', 'the', 'the', 'the', 'the']
Actual:  ['<sos>', 'it', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Loss:  8.810749053955078
Predict:  ['<sos>', '<sos>', 'the', 'the', 'the', 'two', 'the', 'the', 'the', 'freaky']
Actual:  ['<sos>', 'it', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Loss:  4.840537643432617
Predict:  ['<sos>', '<sos>', 's', 'just', 'these', 'two', 'the', 'the', 'the', 'freaky']
Actual:  ['<sos>', 'it', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Loss:  1.9151222229003906
Predict:  ['<sos>', '<sos>', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Actual:  ['<sos>', 'it', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Loss:  0.9088132858276368
Predict:  ['<sos>', 'it', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Actual:  ['<sos>', 'it', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Loss:  0.5086045742034913
P

KeyboardInterrupt: 

In [46]:
# for i in range(1000):
#     train(overfit_vi_train, overfit_en_train, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, BATCH_SIZE)

Training Loss:  8.852307891845703
Predict:  ['<sos>', '<sos>', 'the', 'just', 'these', 'the', 'notes', 'the', 'the', 'the']
Actual:  ['<sos>', 'it', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Training Complete
Training Loss:  4.9451759338378904
Predict:  ['<sos>', '<sos>', 'coming', 'just', 'these', 'notes', 'notes', 'the', 'the', 'the']
Actual:  ['<sos>', 'it', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Training Complete
Training Loss:  1.9719776153564452
Predict:  ['<sos>', '<sos>', 's', 'just', 'these', 'two', 'notes', 'the', 'the', 'freaky']
Actual:  ['<sos>', 'it', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Training Complete
Training Loss:  0.9234449386596679
Predict:  ['<sos>', '<sos>', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'freaky']
Actual:  ['<sos>', 'it', 's', 'just', 'these', 'two', 'notes', 'in', 'the', 'middle']
Training Complete
Training Loss:  0.5161348819732666
Predict:  ['<sos>', 'it', 's', 'just', 'thes

KeyboardInterrupt: 

In [ ]:
# def train(train_input, train_target, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, batch_size):
#     # Batch
#     total_avg_loss = 0
#     for i in range(len(train_input) // batch_size):
#         loss = 0
#         encoder_hidden = encoder.initHidden()
        
#         batch = get_batch(i, batch_size, train_input, train_target)
#         # size batch_size * seq_length
#         batch_input = torch.tensor(batch[0], device=device)
#         batch_target = torch.tensor(batch[1], device=device)
#         input_length = batch_input.shape[1] ## should be seq length
#         target_length = batch_target.shape[1]
#         print(input_length, target_length)

#         encoder_optimizer.zero_grad()
#         decoder_optimizer.zero_grad()
        
#         encoder_outputs = torch.zeros(input_length, batch_size, 1, 256, device=device)
#         encoder_hiddens = torch.zeros(input_length, 1, batch_size, 256, device=device)
        
#         # Encode
#         for ec_idx in range(input_length):
#             # input batch_size * 1
#             encoder_output, encoder_hidden = encoder(batch_input[:, ec_idx].unsqueeze(1), encoder_hidden)
#             encoder_outputs[ec_idx] = encoder_output
#             encoder_hiddens[ec_idx] = encoder_hidden
        
#         # Decode
#         decoder_input = torch.tensor([2] * batch_size, device=device) # SOS token 2
#         decoder_hidden = encoder_hidden
        
#         ## Print Value
#         sample_sentence = []
        
#         # Always use Teacher Forcing
#         for dc_idx in range(target_length):
#             decoder_output, decoder_hidden = decoder(decoder_input.unsqueeze(1), decoder_hidden)
#             decoder_output = decoder_output.squeeze(1).to(device) # get rid of the seq dimention
#             loss += criterion(decoder_output, batch_target[:, dc_idx])
#             decoder_input = batch_target[:, dc_idx]
            
#             ## Print Value
#             sample_sentence.append(torch.argmax(decoder_output[0]).item())
            
#         loss.backward()
#         total_avg_loss += loss.item() / target_length
        
#         encoder_optimizer.step()
#         decoder_optimizer.step()
        
# #         print('Training Loss: ', loss.item() / target_length)
        
#         ## Print Value
#         print("Predict: ", ids2sentence(sample_sentence, id2word_en_dic))
#         print("Actual: ", ids2sentence(batch_target[0].cpu().numpy(), id2word_en_dic))
        
#     return total_avg_loss